In [ ]:
import pandas as pd
import numpy as np
#
import faiss
import pyterrier as pt

In [ ]:
if not pt.started():
    pt.init()

In [ ]:
# Intersección
def list_intersection(l1, l2):
    s1 = set(l1.tolist()[0])
    s2 = set(l2.tolist()[0])
    return len(s1.intersection(s2)) / len(s1)

---
### MAIN
---

In [ ]:
dataset = pt.get_dataset("vaswani")
print("Corpus Vaswani: %s " % dataset.get_corpus())

In [ ]:
documents = pd.DataFrame(dataset.get_corpus_iter())
documents.shape

In [ ]:
documents.head()

In [ ]:
#!pip install transformers
#!pip install sentence-transformers
#from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

#doc_text = list(documents['text'].head(10))
#demo_embeddings = model.encode(doc_text, convert_to_tensor=True)
#print (demo_embeddings.shape)
#
#demo_embeddings

In [ ]:
# Cargo los embeddings de todos los docs previamente calculados.
vaswani_docs_embeddings = np.load("../data/vaswani_docs_embeddings-512.npy")
vaswani_docs_embeddings.shape

In [ ]:
vaswani_docs_embeddings[1]

In [ ]:
# Obtengo los 'topics' asociados al corpus
topics = dataset.get_topics()
topics.head()

In [ ]:
# Cargo los embeddings de todos los queries previamente calculados.
vaswani_query_embeddings = np.load("../data/vaswani_query_embeddings-512.npy")
vaswani_query_embeddings.shape

### Indexación con FAISS (diferentes índices)
**Más sobre los tipos de índices en FAISS:** https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

**Prueba 1 - Flat Index**  
Recordar: En este tipo de índice se mide la distancia L2 (euclídea) entre el vector de query 
y todos los vectores de documentos almacenados. Es simple y preciso (pero no demasiado rápido).

In [ ]:
# Inicialización
d = 512
indexFlat = faiss.IndexFlatL2(d)

# Chequeo cantidad de docs en el índice
indexFlat.ntotal

In [ ]:
# Agrego los documentos al índice
%time
indexFlat.add(vaswani_docs_embeddings)
indexFlat.ntotal

In [ ]:
indexFlat.is_trained

In [ ]:
# Ejemplo de recuperación
k = 4
query_vector = np.array([vaswani_query_embeddings[0]])

In [ ]:
%time
DFlat, rsFlat = indexFlat.search(query_vector, k)  # Búsqueda

In [ ]:
print(rsFlat)

In [ ]:
DFlat

In [ ]:
faiss.write_index(indexFlat, "vaswani_faiss_flat.ndx")

**Prueba 2 - IVF Flat Index**  
Recordar: En este tipo de índice se particiona el espacio de búsqueda (nlist) para realizar
un ANN.

In [ ]:
nlist = 50  # Cantidad de celdas
quantizer = faiss.IndexFlatL2(d)
indexIVFFlat = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
indexIVFFlat.is_trained

In [ ]:
# Preparo (train) las estructuras de datos del índice
%time
indexIVFFlat.train(vaswani_docs_embeddings)
indexIVFFlat.ntotal

In [ ]:
indexIVFFlat.is_trained

In [ ]:
# Agrego los documentos al índice
indexIVFFlat.add(vaswani_docs_embeddings)
indexIVFFlat.ntotal

In [ ]:
# Ejemplo de recuperación
k = 5
query_vector = np.array([vaswani_query_embeddings[0]])

In [ ]:
%time
DIVFFlat, rsIVFFlat = indexIVFFlat.search(query_vector, k)  # Búsqueda

In [ ]:
rsIVFFlat

In [ ]:
list_intersection(rsFlat, rsIVFFlat)

In [ ]:
indexIVFFlat.nprobe = 10 # Aumentamos el ámbito de búsqueda a 10 celdas
DIVFFlat, rsIVFFlat = indexIVFFlat.search(query_vector, k)  # Búsqueda

In [ ]:
rsIVFFlat

In [ ]:
faiss.write_index(indexIVFFlat, "vaswani_faiss_ivfflat.ndx")

**Prueba 3 - IVF Flat Index con Product Quantization**  
Recordar: En este tipo de índice se aplica Product Quantization para reducir (comprimir) los vectores. 
Luego se ejecuta ANN.

In [ ]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
indexIVFPQ = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [ ]:
indexIVFPQ.is_trained

In [ ]:
# Preparo (train) las estructuras de datos del índice
%time
indexIVFPQ.train(vaswani_docs_embeddings)
indexIVFPQ.ntotal

In [ ]:
# Agrego los documentos al índice
indexIVFPQ.add(vaswani_docs_embeddings)
indexIVFPQ.ntotal

In [ ]:
# Ejemplo de recuperación
k = 5
query_vector = np.array([vaswani_query_embeddings[0]])

In [ ]:
%time
DIVFPQ, rsIVFPQ = indexIVFPQ.search(query_vector, k)  # Búsqueda

In [ ]:
rsIVFPQ

In [ ]:
indexIVFPQ.nprobe = 10
%time
DIVFPQ, rsIVFPQ = indexIVFPQ.search(query_vector, k)  # Búsqueda

In [ ]:
rsIVFPQ

In [ ]:
faiss.write_index(indexIVFPQ, "vaswani_faiss_ivfpq.ndx")

### Tarea 
**Ejecutar todo el set de consultas y calcular el solapamiento promedio de las listas de 
resultados para nprobe = [1 .. 10]**